# Problem III

<b>Please create a simple machine learning pipeline</b> that will give user recommendation (1 user at least has 100 product recommendation) and save the recommendation to PostgreSQL using this <a href='https://nijianmo.github.io/amazon/index.html'>Amazon Dataset</a> and upload it in your github page. You will be grade not for the machine learning algorithm / model but for the pipeline performance (eg. processing time, no of data processed).

## Brief Introducttion
Recommendation systems have been widely used in product recommendations such as in Netflix, Amazon, Spotify, etc. In general, there are 2 methods for building a basic recommender systems:
1. Collaborative Filtering
2. Content-based Filtering

The method/algorithm I am going to use here is collaborative filtering. Collaborative filtering filters the interest of a user on a specific product by gathering/collecting information from other similar users.<br>
`If person A shares a similar opinion with person B, then person A is more likely to share person B's opinion on another issue than a random person`<br>
On the other hand, content-based filtering makes use of features/attributes to recommend products similar to the user liking based on other users' previous actions or explicit feedbacks.<br><br>

I will use the Amazon dataset provided from the link above, but i only use the 'Beauty Products' which contains about 370k rows for cheap and faster implementation. (The whole dataset is about 233.1 million records, which is roughly 34GB)

## Solution

### 1. Load and Preprocess Dataset
Since the dataset is in .gzip format, use gzip and json libraries from Python to load the dataset

In [1]:
import numpy as np
import pandas as pd
import gzip # read .gzip files
import json # take care of json formatted contents

In [2]:
def parse(path):
    """
    Read a .gzip file and parse its JSON contents
    
    # Arguments:
        path: string, absolute path to the dataset
        
    # Outputs:
        json_content: dict, the parsed JSON content
    
    """
    
    g = gzip.open(path, 'rb')
    for l in g:
        json_content = json.loads(l)
        
        yield json_content

def getDF(path):
    """
    Create a DataFrame from JSON content
    
    # Arguments:
        path: string, absolute path to the dataset
    
    # Returns:
        df: pandas.DataFrame, the loaded dataset in a DataFrame format
    
    """
    
    i = 0
    df = {}
    
    for d in parse(path):
        df[i] = d
    
        i += 1
      
    return pd.DataFrame.from_dict(df, orient='index')

In [3]:
# load dataset
df = getDF('data/All_Beauty.json.gz')

print("Number of observations:", len(df))
print('First 5 rows:')
df.head()

Number of observations: 371345
First 5 rows:


,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,1.0,True,"02 19, 2015",A1V6B6TNIC10QE,0143026860,theodore j bigham,great,One Star,1424304000,NaN,NaN,NaN
1,4.0,True,"12 18, 2014",A2F5GHSXFQ0W6J,0143026860,Mary K. Byke,My husband wanted to reading about the Negro ...,... to reading about the Negro Baseball and th...,1418860800,NaN,NaN,NaN
2,4.0,True,"08 10, 2014",A1572GUYS7DGSR,0143026860,David G,"This book was very informative, covering all a...",Worth the Read,1407628800,NaN,NaN,NaN
3,5.0,True,"03 11, 2013",A1PSGLFK1NSVO,0143026860,TamB,I am already a baseball fan and knew a bit abo...,Good Read,1362960000,NaN,NaN,NaN
4,5.0,True,"12 25, 2011",A6IKXKZMTKGSC,0143026860,shoecanary,This was a good story of the Black leagues. I ...,"More than facts, a good story read!",1324771200,5,NaN,NaN


According to the official website, each column headers tell us about:
<ul>
    <li>reviewerID - ID of the reviewer, e.g. A2SUAM1J3GNN3B</li>
    <li>asin - ID of the product, e.g. 0000013714</li>
    <li>reviewerName - name of the reviewer</li>
    <li>vote - helpful votes of the review</li>
    <li>style - a disctionary of the product metadata, e.g., "Format" is "Hardcover"</li>
    <li>reviewText - text of the review</li>
    <li>overall - rating of the product</li>
    <li>summary - summary of the review</li>
    <li>unixReviewTime - time of the review (unix time)</li>
    <li>reviewTime - time of the review (raw)</li>
    <li>image - images that users post after they have received the product</li>
</ul>

Meanwhile, our workflow can be described as follows:
1. A user rates an item/a product
2. Ratings are considered as the user's level of interest/satisfaction on a specific item
3. The system will find similar ratings from differrent users against a user
4. Finally, the system recommends item(s) that have high ratings, but not yet rated by this user

Thus, according to the workflow above, we can conclude that `overall` and `reviewerName` are the only columns that have high impact in training our recommendation system model.

### 2. Explore and Analyze Dataset
Let's explore and analyze the data a little bit. Pandas has provided some useful functions that I can utilize

In [4]:
df.describe()

,overall,unixReviewTime
count,371345.000000,3.713450e+05
mean,4.112093,1.440742e+09
std,1.362099,6.962743e+07
min,1.000000,9.474624e+08
25%,4.000000,1.416096e+09
50%,5.000000,1.456963e+09
75%,5.000000,1.485562e+09
max,5.000000,1.538438e+09


`object` data types can be in form of strings, lists, dictionaries, etc.

In [5]:
df.dtypes

overall           float64
verified             bool
reviewTime         object
reviewerID         object
asin               object
reviewerName       object
reviewText         object
summary            object
unixReviewTime      int64
vote               object
style              object
image              object
dtype: object

Since `overall` column tells us about user ratings, we have to make sure that there are no empty/NaN values.

In [6]:
print("Overall empty values count:", np.sum(pd.isnull(df['overall'])))

Overall empty values count: 0


Great, we can see that the output '0' means we have no empty values inside `overall` column.<br>
How about other columns?

In [7]:
# count how many empty values each column has
for c in df.columns:
    if c == 'overall':
        continue
    print("{} empty values count: {}".format(c, np.sum(pd.isnull(df[c]))))

verified empty values count: 0
reviewTime empty values count: 0
reviewerID empty values count: 0
asin empty values count: 0
reviewerName empty values count: 38
reviewText empty values count: 399
summary empty values count: 206
unixReviewTime empty values count: 0
vote empty values count: 319446
style empty values count: 245387
image empty values count: 362954


There are some empty/NaN values here and there. Since the only columns we care about are `overall` and `reviewerName`, we can drop them later.<br>
Nevertheless, it is a great common practice to fill any empty values with a default value (e.g. empty string `''`)

In [8]:
df = df.fillna('') # fill empty/NaN values with empty strings ('')
df = df.drop(['verified', 'reviewTime', 'unixReviewTime', 'vote', 'style', 'image'], axis=1) # drop unused columns

Let's check the final preprocessed and cleaned dataset

In [9]:
df.head()

,overall,reviewerID,asin,reviewerName,reviewText,summary
0,1.0,A1V6B6TNIC10QE,0143026860,theodore j bigham,great,One Star
1,4.0,A2F5GHSXFQ0W6J,0143026860,Mary K. Byke,My husband wanted to reading about the Negro ...,... to reading about the Negro Baseball and th...
2,4.0,A1572GUYS7DGSR,0143026860,David G,"This book was very informative, covering all a...",Worth the Read
3,5.0,A1PSGLFK1NSVO,0143026860,TamB,I am already a baseball fan and knew a bit abo...,Good Read
4,5.0,A6IKXKZMTKGSC,0143026860,shoecanary,This was a good story of the Black leagues. I ...,"More than facts, a good story read!"


Looking good! We can proceed with the next step.<br>
Next, for flexibility purposes, I store all users along with their names and UIDs in a `dict` called 'users'. This `dict` will makes the prediction phase easier (which I will explain later)

In [10]:
users = {} # dict of users

# fill users dict according to their names and UIDs
for name,uid in zip(df['reviewerName'], df['reviewerID']):
    if name not in users:
        users[name] = uid

# display the first 20 users
for i, n in enumerate(df['reviewerName']):
    if i == 20:
        break
    print(n, users[n])
    

theodore j bigham A1V6B6TNIC10QE
Mary K. Byke A2F5GHSXFQ0W6J
David G A1572GUYS7DGSR
TamB A1PSGLFK1NSVO
shoecanary A6IKXKZMTKGSC
W. Powell A36NF437WZLQ9E
Robert S. Clay Jr. A10Q8NIFOVOHFV
Jacqueline Diaz A26PO1B2Q2G1CS
Khadijah Ali-Evans AQ812VYVTC2RJ
rabiyaa123 AMACNEW14ADMX
Lilly A5FS4FVWR77O6
igzotikvet A1NLF2JD7BVOB4
M. Morretti A3CKVF2ZR1CBO0
Amazon Customer A3IFT6SR16SAYU
Zury M. A2BQ7NB90SBVIA
Shenia Morris A92AYWTA52KXE
Amazon Customer A3IFT6SR16SAYU
Alex G A2V9BG2MDQVCYX
TSENG TSUNG-CHIN AIL9Q82L1G2YO
Nikolai G. J. Geier A1ROGN2QPKZGP7


### 3. Model Training
We will use `scikit-surprise` to implement colaborative filtering and train our model

In [11]:
# Uncomment and run code below to install scikit-surprise
#!pip installl scikit-surprise

In [12]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection.validation import cross_validate

In [13]:
reader = Reader() # content parser inside the dataset
data = Dataset.load_from_df(df[['reviewerID', 'asin', 'overall']], reader) # load our DataFrame 'df' dataset to be fed to the model
svd_model = SVD() # our model instance, built with SVD. We can also build different models like Probabilistic Matrix Factorization (PMF) or Non-negative Matrix Factorization (NMF)

`Reader()` class will parse the file containing users, products, and ratings.<br>
Once the model is ready, we can evaluate the model by implementing k-fold cross-validation.

In [14]:
# Run 5-fold cross-validation and print results
cross_validate(svd_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2530  1.2559  1.2565  1.2577  1.2501  1.2546  0.0027  
MAE (testset)     0.9708  0.9714  0.9734  0.9730  0.9680  0.9713  0.0019  
Fit time          12.99   12.99   13.14   13.34   13.27   13.15   0.14    
Test time         0.31    0.31    0.33    0.33    0.33    0.32    0.01    


{'test_rmse': array([1.25303686, 1.25587946, 1.25649284, 1.25765975, 1.25006423]),
 'test_mae': array([0.97080472, 0.97135474, 0.97337209, 0.97301144, 0.96803664]),
 'fit_time': (12.990058660507202,
  12.994434118270874,
  13.136275053024292,
  13.344036102294922,
  13.270441770553589),
 'test_time': (0.308274507522583,
  0.31319665908813477,
  0.3268883228302002,
  0.3254354000091553,
  0.33092427253723145)}

We can also retrain our model with the entire training dataset

In [15]:
new_training_set = data.build_full_trainset()
svd_model.fit(new_training_set)

### 4. Model Prediction
Before we get into the predicting step, we need to load the metadata dataset because this dataset contains all products' name, description, brand, price, etc.

In [16]:
# load metadata dataset
metadata = getDF('data/meta_All_Beauty.json.gz')

In [17]:
# shows top 5 rows
metadata.head()

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",[],,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],[https://images-na.ssl-images-amazon.com/image...,,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],[https://images-na.ssl-images-amazon.com/image...,,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790


For simplicity, I only show the 'product ID' (asin) and 'product name' (title) in the prediction phase

In [18]:
metadata = metadata[['asin', 'title']] # extract only product id and product name
metadata.head()

,asin,title
0,6546546450,Loud 'N Clear&trade; Personal Sound Amplifier
1,7178680776,No7 Lift &amp; Luminate Triple Action Serum 50...
2,7250468162,No7 Stay Perfect Foundation Cool Vanilla by No7
3,7367905066,Wella Koleston Perfect Hair Colour 44/44 Mediu...
4,7414204790,Lacto Calamine Skin Balance Oil control 120 ml...


Recall that I made a `dict` called 'users' in the beginning of this notebook and this is where it gets very useful. On training phase, the model only recognizes UID (users' ID) instead of names. Hence, when making a prediction, we also need to feed in the user ID. 'users' `dict` will make our job easier so that we can enter a name and it will find its corresponding ID

In [19]:
USER_NAME = 'Shenia Morris' # user name
USER_ID = users[USER_NAME] # user ID

# append 'recommendation_score' column in order to a make prediction
metadata['recommendation_score'] = metadata['asin'].apply(lambda x: svd_model.predict(USER_ID, x).est)

In [20]:
num_of_recommendations_to_show = 100 # show top 100 recommendations
metadata = metadata.sort_values(by=['recommendation_score'], ascending=False) # sort by descending order (big to small recommendation_score)

print('Top {} Recommendations for {}:'.format(num_of_recommendations_to_show, USER_NAME))
recommendations = metadata.head(num_of_recommendations_to_show)
recommendations

Top 100 Recommendations for Shenia Morris:


,asin,title,recommendation_score
1206,B000F8JKF8,EO Essential Oil Products Liquid Hand Soap Ros...,5.000000
11920,B00HS5BGO4,Green Soap 8oz Spray Bottle (Multiple Pack Opt...,5.000000
4036,B002ELYUNG,Cera di Cupra Rosa Face Cream (Dry Skin Formul...,5.000000
697,B00021DJ32,"NARS Blush, Taj Mahal",5.000000
26295,B0195R1FT8,Booty Magic | Butt Enhancement Cream - 2 Month...,5.000000
...,...,...,...
14228,B00KQ12A0G,The Doctor's BrushPicks | Interdental Toothpic...,4.864380
29151,B01CTLTOHM,"Unitrex - Aromafier Diffuser Refill Pads, (3-P...",4.863848
3887,B0028K3IWM,"Gillette Clinical Strength Advanced Solid, All...",4.863734
4465,B003EERKK2,Zest Aqua 4 oz Bar Soaps - 16 Bars,4.863618


### 5. Saving Prediction
Finally, save those predictions (recommendation list) into a Postgresql database. To do this, we need to install `psycopg2` library and setup a local database (https://www.postgresql.org/download/)

In [21]:
# Uncomment and run this code below to install psycopg2 which can connect Python to Postgresql
#!pip install psycopg2

To make it more convenient, we can also make a configuration file to store connection parameters. In practice, developers often use `database.ini` which contains:

```
[postgresql]
host=localhost
database=db_name
user=postgres
password=admin
```

By doing this, we can change the configuration connection parameters when we move the code to production environment without making changes to the code

In [22]:
import psycopg2

def insert_recommendation_list(product_id_list, product_name_list):
    
    """
    Insert records (recommendation lists) into a Postgresql database
    
    # Arguments:
        product_id_list: list, a list of product IDs
        product_name_list: list, a list of product names
    
    # Outputs:
        None
    """
    
    # SQL INSERT query
    query = """INSERT INTO recommendations(product_id_list, product_name_list)
            VALUES(%s, %s)"""
    
    conn = None # set initial connection to null
    
    # initiate connection to database
    try:
        print('Connecting to database...')
        conn = psycopg2.connect(host='localhost', # local connection
                               database='kitabisa_db', # our DB name
                               user='admin', # username
                               password='k1t@B!$a') # password
        cur = conn.cursor()
        print('Inserting rows...')
        cur.execute(sql, (product_id_list, product_name_list)) # execute INSERT command
        conn.commit() # make changes to the database
        cur.close()
    
    # throw an error if connection to database cannot be initiated
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    
    # close database connection
    finally:
        if conn is not None:
            print('Transaction completed, closing database connection...')
            conn.close()

In [23]:
insert_recommendation_list(recommendations['asin'].values.tolist(), recommendations['title'].values.tolist())